# R_2 Through EDA of 65 GB tweets (on Politics)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("COMP90024_A2_EDA")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/23 17:06:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read In Original
spark_json = spark.read.json('../data/raw/BigTwitterFile/twitter-huge.json')

In [3]:
# Observe Schema
spark_json.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- doc: struct (nullable = true)
 |    |-- _id: string (nullable = true)
 |    |-- _rev: string (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- context_annotations: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- domain: struct (nullable = true)
 |    |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- entity: struct (nullable = true)
 |    |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- entiti

### Extract Key Columns

In [4]:
# important features to extract out of the original dataframe
out_df = spark_json.select('doc._id', 'doc.data.created_at', 'doc.data.geo.coordinates.coordinates', 
                  'doc.data.geo.coordinates.type', 'doc.includes', 
                  'doc.data.geo.place_id', 'doc.data.lang', 'doc.data.sentiment', 'doc.data.text', 
                  'doc.data.author_id')

### Missing Geolocation

In [5]:
# observe missing values in 'includes' (geolocation)


from pyspark.sql.functions import col, sum

def count_missing_values(df, column_name):
    # Filter the DataFrame to select rows where the column is null
    filtered_df = df.filter(col(column_name).isNull())
    
    # Use the `sum()` function to count the number of rows with null values
    count = filtered_df.select(sum(col(column_name).isNull().cast("int"))).collect()[0][0]
    
    return count

missing_count = count_missing_values(out_df, "includes")

In [64]:
missing_count

49300230

In [65]:
out_df.count()

52533743

In [66]:
52533743 - 49300230

3233513

### Extract Tweets containing AI Keywords

In [6]:
import pandas as pd
words_df = pd.read_excel('../data/raw/Keywords/Voting Keyword.xlsx')

words = list(words_df['keywords'])
words

['accountab',
 'advoca',
 'affirmative action',
 'agenda setting',
 'aspiration',
 'asylum seeker',
 'bureaucra',
 'business',
 'citizen',
 'class',
 'colon',
 'committee',
 'communit',
 'conservati',
 'constitution',
 'convention',
 'corrupt',
 'crisis',
 'cultur',
 'culture wars',
 'federalism',
 'democracy',
 'differen',
 'discourse',
 'economic rationalis',
 'egalitarian',
 'elit',
 'environment',
 'equal',
 'ethic',
 'ethinic',
 'executive',
 'famili',
 'feminis',
 'free',
 'gender',
 'globalis',
 'green',
 'identity',
 'ideolog',
 'indigenous',
 'individual',
 'interest group',
 'international relations',
 'internet',
 'judicia',
 'law',
 'leader',
 'liberal',
 'magnagerialism',
 'mandate',
 'media',
 'ministerialadvisers',
 'minist ',
 'multicultural',
 'national identity',
 'national identity',
 'obligation',
 'others',
 'otherness',
 'parliament',
 'participation',
 'party',
 'parties',
 'polic',
 'political socialisation',
 'populism',
 'post-modern',
 'power',
 'protest',
 '

In [7]:
# Only get tweets that contain AI related keywords

from pyspark.sql.functions import col

# Assume 'df' is your DataFrame with a string typed column named 'text'
filter_cond = col('doc.data.text').contains(words[0])  # initial filter condition

for i in range(1, len(words)):
    filter_cond = filter_cond | col('text').contains(words[i])  # add each snippet to the filter condition using the & operator

filtered_df = out_df.filter(filter_cond)  # apply the filter condition to the DataFrame

result_df = filtered_df.filter(col("doc.includes").isNotNull())

In [8]:
result_df.count()

319631

In [9]:
# convert to Pandas df
df = result_df.select("*").toPandas()

In [11]:
df

,_id,created_at,coordinates,type,includes,place_id,lang,sentiment,text,author_id
0,1491795863018274816,2022-02-10T15:27:04.000Z,None,None,"{""places"":[{""full_name"":""Brisbane, Queensland""...",004ec16c62325149,en,-0.142857,@IMmadashellandi @HeshmatAlavi Israel is a dem...,4516171573
1,1491725195375439877,2022-02-10T10:46:16.000Z,None,None,"{""places"":[{""full_name"":""Brisbane, Queensland""...",004ec16c62325149,en,-0.200000,"@stilllukebowden More power to Barbara, Sue an...",177543302
2,1491648779082100739,2022-02-10T05:42:37.000Z,None,None,"{""places"":[{""full_name"":""Sydney, New South Wal...",0073b76548e5984f,en,-0.015152,@Dr_eVal @Sg3912 @parsleysmum @ardentgreeneyes...,987281520099344384
3,1491626908106190848,2022-02-10T04:15:42.000Z,None,None,"{""places"":[{""full_name"":""Melbourne, Victoria"",...",01864a8a64df9dc4,en,0.333333,@sammyjcomedian @NikkiKlopfer Another reclaime...,418231895
4,1491588166888742912,2022-02-10T01:41:46.000Z,None,None,"{""places"":[{""full_name"":""Melbourne, Victoria"",...",01864a8a64df9dc4,en,0.000000,@MattGurchenko @MrInbetween1 @Garyandthegoat1 ...,1466177366506549251
...,...,...,...,...,...,...,...,...,...,...
319626,1557514633623191553,2022-08-10T23:50:00.000Z,None,None,"{""places"":[{""full_name"":""Brisbane, Queensland""...",004ec16c62325149,en,-0.023810,Trump Attorney wasn’t allowed presence when KG...,1511175941846102021
319627,1557515011781640193,2022-08-10T23:51:30.000Z,None,None,"{""places"":[{""full_name"":""Laidley, Queensland"",...",00397523196f3174,en,0.250000,Mod parties be like https://t.co/jm21gF9qIy,803641382
319628,1557515835237072896,2022-08-10T23:54:47.000Z,None,None,"{""places"":[{""full_name"":""Plenary Hall"",""geo"":{...",07d9e72979884000,en,0.051282,"""Teacher's need to start where they're comfort...",1074293586
319629,1557515963662077953,2022-08-10T23:55:17.000Z,None,None,"{""places"":[{""full_name"":""Brisbane, Queensland""...",004ec16c62325149,en,-0.037037,Laid back sleeping ordinary undecided conserv...,1511175941846102021


In [ ]:
len(df)

In [56]:
# number of 'real AI related' tweets and has useable geolocation
len(df[(~df['includes'].isnull())])

49952

In [12]:
# observes some of these real AI related tweet

i = 0
for txt in df[(~df['includes'].isnull())]['text']:
    print(txt)
    print('==')
    i += 1

    if i == 200:
        break

# HOWEVER, DOESN'T SEEM TO HAVE ACTUALLY MATCHED

@IMmadashellandi @HeshmatAlavi Israel is a democracy and not a dictatorship ruled by fanatics that threatens death to America and Israel! Crawl out of your cave and see light!
==
@stilllukebowden More power to Barbara, Sue and Sarah I say!
==
@Dr_eVal @Sg3912 @parsleysmum @ardentgreeneyes @AlanSla90124663 @HHazcat @Freeds @catmum1932 @MariaPulk @kenobe_wan @denvor18 @Nacho87329549 @Merlin80643296 @starbuckssue @Leonlovescats @tiggy_catt247 @MarisaBerlinge1 @3phibotticelli @LMEVegetarian @GeorgeandMildr4 @JLHerms @cat_fender @EdibleCatsClub @maureenelis @LittleTess3 @creative54 @azzurro_azzurro @NeddlePeddle @TriciaCaputo @walphieplopflee @JenniferBMorri2 @Margare72877928 @MirMGreg @lolamatt47 @MistyKitty3 @NatsRallyCat @Gabrielle_Hope1 @NatalieRobins7 @TheVicarageCat @Wxgirl169 @linda_austin1 @_mongoose @Mrs_J_Bananas @A_affirmations @Meow_Factor @PussycatNaughty @MelvinJ75085444 @AndSalume No poor Etty !!!. Max is fine !
==
@sammyjcomedian @NikkiKlopfer Another reclaimed table. Thanks

23/04/23 18:53:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 359495 ms exceeds timeout 120000 ms
23/04/23 18:53:09 WARN SparkContext: Killing executors is not supported by current scheduler.


In [10]:
df.to_csv('../data/semi_cleaned_voting_tweets.csv')

In [1]:
import pandas as pd
df = pd.read_csv('../data/semi_cleaned_voting_tweets.csv')

In [3]:
df.loc[0]['includes']

'{"places":[{"full_name":"Brisbane, Queensland","geo":{"type":"Feature","bbox":[152.668522848,-27.767440994,153.31787024,-26.996844991],"properties":{}},"id":"004ec16c62325149"}]}'

# OTHER EDA (NOT IMPORTANT)

In [24]:
coord = filtered_df['coordinates']
coord = [x for x in coord if x != None]
len(coord)

4093

In [26]:
type = filtered_df['type']
type = [x for x in type if x != None]

len(type)

2

In [28]:
place_id = filtered_df['place_id']
place_id = [x for x in place_id if x != None]

len(place_id)

62422

In [50]:
place_id

['0073b76548e5984f',
 '01864a8a64df9dc4',
 '0ea2a2fc1c54f000',
 '0118c71c0ed41109',
 '008402196cea8a83',
 '0118c71c0ed41109',
 '01864a8a64df9dc4',
 '01e8a1a140ccdc5c',
 '0023f2ab28f1c387',
 '0073b76548e5984f',
 '01864a8a64df9dc4',
 '01864a8a64df9dc4',
 '01e8a1a140ccdc5c',
 '0118c71c0ed41109',
 '01864a8a64df9dc4',
 '01e4b0c84959d430',
 '01864a8a64df9dc4',
 '017453ae077eafd3',
 '004ec16c62325149',
 '017453ae077eafd3',
 '017453ae077eafd3',
 '017453ae077eafd3',
 '017453ae077eafd3',
 '017453ae077eafd3',
 '004ec16c62325149',
 '01864a8a64df9dc4',
 '01864a8a64df9dc4',
 '01864a8a64df9dc4',
 '019e32e73d7d3282',
 '01864a8a64df9dc4',
 '00a515d443f09983',
 '01864a8a64df9dc4',
 '01864a8a64df9dc4',
 '01864a8a64df9dc4',
 '01864a8a64df9dc4',
 '01e4b0c84959d430',
 '0073b76548e5984f',
 '004ec16c62325149',
 '004ec16c62325149',
 '01864a8a64df9dc4',
 '0073b76548e5984f',
 '01864a8a64df9dc4',
 '0073b76548e5984f',
 '265b933184e2b88a',
 '0073b76548e5984f',
 '0073b76548e5984f',
 '009b89410157afec',
 '0073b76548e

In [49]:
filtered_df[(filtered_df['ai_bool'] == 1) & (filtered_df['coordinates'].notnull())]

,_id,created_at,coordinates,type,type,place_id,lang,sentiment,text,author_id,ai_bool
4972,1491759637648928768,2022-02-10T13:03:07.000Z,"[77.19102783, 28.5974128]",Point,Point,317fcc4b21a604d5,en,0.025641,INNR — AI patient room automation introduced i...,149585330,1
318371,1511008845765644290,2022-04-04T15:52:36.000Z,"[133.96339777, -25.63642933]",Point,Point,40888ddb9f11c18f,en,-0.130435,Online abuse targeting footballers to be tackl...,1491316688549253120,1
380149,1514717036705845252,2022-04-14T21:27:38.000Z,"[-73.60404689, 45.51949281]",Point,Point,36775d842cbec509,ko,0.000000,"TW : ""귀사의 AI는 안녕하십니까?"" NIST, AI 위험 관리 프레임워크 마...",149585330,1
739171,1535990896960819200,2022-06-12T14:22:21.000Z,"[144.94161989, -37.78821679]",Point,Point,01864a8a64df9dc4,en,-0.043478,Google Engineer On Leave After He Claims AI Pr...,1491316688549253120,1
745330,1536394754170097664,2022-06-13T17:07:08.000Z,"[144.95697498, -37.7813477]",Point,Point,01864a8a64df9dc4,en,0.035714,"Spotify is acquiring Sonantic, the AI voice pl...",1491316688549253120,1
745332,1536394785245696000,2022-06-13T17:07:16.000Z,"[144.95697498, -37.7813477]",Point,Point,01864a8a64df9dc4,en,-0.111111,#news #gear #internet #services #microsoft Mic...,1491316688549253120,1
787425,1539093299419615234,2022-06-21T03:50:12.000Z,"[151.20081, -33.88339]",Point,Point,0073b76548e5984f,en,0.212121,So excited to be back @utsengineeringandit @th...,1181037025,1
945846,1549830466630406144,2022-07-20T18:55:52.000Z,"[144.95697498, -37.7813477]",Point,Point,01864a8a64df9dc4,en,0.050000,Comet Recognized as a Cool Vendor in AI Core T...,1506620226162634758,1
980571,1551734835886055426,2022-07-26T01:03:09.000Z,"[151.1036, -33.9104]",Point,Point,0073b76548e5984f,en,0.000000,"Since my AI experiments, the hackers have made...",18742857,1
1004798,1553150790432333825,2022-07-29T22:49:39.000Z,"[151.1987, -33.8865]",Point,Point,0073b76548e5984f,en,0.184211,The future is AI - been creating with @openaid...,21818549,1


In [25]:
filtered_df

,_id,created_at,coordinates,type,type,place_id,lang,sentiment,text,author_id
0,1491845931456774147,2022-02-10T18:46:01.000Z,None,None,None,None,en,0.020408,@Real_DrugTalk DL Down3r´s hit inspirational s...,62407961
1,1491846139196452864,2022-02-10T18:46:51.000Z,None,None,None,None,en,0.020408,@Real_DrugTalk DL Down3r´s hit inspirational s...,62407961
2,1491817025433489409,2022-02-10T16:51:10.000Z,None,None,None,None,en,0.020833,@slavetothehat DL Down3r´s hit inspirational s...,62407961
3,1491844343379992579,2022-02-10T18:39:43.000Z,None,None,None,None,en,0.020408,"@jots_8 DL Down3r´s hit inspirational song "" S...",62407961
4,1491814800212611079,2022-02-10T16:42:19.000Z,None,None,None,None,en,0.020833,"@badwool9 DL Down3r´s hit inspirational song ""...",62407961
...,...,...,...,...,...,...,...,...,...,...
1074884,1557516967275528193,2022-08-10T23:59:16.000Z,None,None,None,None,en,0.000000,🇦🇺👏 A great day for the Aussies at the #Canadi...,43806353
1074885,1557517000686968832,2022-08-10T23:59:24.000Z,None,None,None,004ec16c62325149,en,-0.162791,"I'm listening to the ""eDNA monitoring; 'teleh...",876592795963736064
1074886,1557517101694611457,2022-08-10T23:59:48.000Z,None,None,None,None,en,0.000000,GBP/USD\nTARGET LEVEL 1.2233\nSL: (B) 1.2199 4...,1115918534
1074887,1557517147898781701,2022-08-10T23:59:59.000Z,None,None,None,None,en,0.120000,AI tech allows businesses to gather and utilis...,223309438


In [8]:
from pyspark.sql.functions import min, max

# assuming your data is stored in a Spark DataFrame called 'df'
min_datetime = spark_json.agg(min("doc.data.created_at")).collect()[0][0]
max_datetime = spark_json.agg(max("doc.data.created_at")).collect()[0][0]

print("Minimum datetime:", min_datetime)
print("Maximum datetime:", max_datetime)

Minimum datetime: 2022-02-10T00:00:00.000Z
Maximum datetime: 2022-08-10T23:59:59.000Z


In [9]:
from pyspark.sql.functions import count, col

n_geo_rows = spark_json.select(count(col('doc.data.geo')).alias('count')).collect()[0]['count']

In [12]:
n_geo_rows

52396417